In [2]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI
from langchain.llms import OpenAI, AzureOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [3]:
chat = ChatOpenAI(temperature=.7)

In [10]:
chat([HumanMessage(content="Hello, how are you?")])

AIMessage(content="Hello! I'm an AI language model, so I don't have feelings, but I'm here to help you. How can I assist you today?", additional_kwargs={}, example=False)

In [13]:
from langchain.document_loaders import NotebookLoader

loader = NotebookLoader(
    "/Users/davidkuchelmeister/Documents/projects_coding/llm/src/llm/google_drive.ipynb",
    include_outputs=True,
    max_output_length=20,
    remove_newline=True,
)
loader.load()

[Document(page_content='\'markdown\' cell: \'[\'---\', \'# Google Drive\', \'\', \'Manual: https://python.langchain.com/docs/modules/data_connection/document_loaders/integrations/google_drive\']\'\n\n \'code\' cell: \'[\'from langchain.document_loaders import GoogleDriveLoader\', \'from langchain.document_loaders import UnstructuredFileIOLoader\', \'from pathlib import Path\']\'\n\n \'code\' cell: \'[\'folder_id = "1KKCZgDYRNqLD-CYotYSfqxaSBb3QPJfB"\', \'loader = GoogleDriveLoader(\', \'    folder_id=folder_id,\', \'    file_loader_cls=UnstructuredFileIOLoader,\', \'    file_loader_kwargs={"mode": "elements"},\', \'    service_account_key=Path("/Users/davidkuchelmeister/Documents/projects_coding/llm/original-frame-289508-7b253fd94c34.json"),\', \')\']\'\n\n \'code\' cell: \'[\'docs = loader.load()\']\'\n\n \'code\' cell: \'[\'from langchain.document_loaders import GoogleDriveLoader\', \'from langchain.document_loaders import UnstructuredFileIOLoader\', \'\', \'file_id = "1evZdMhMwWGfgX

---
# Chat History

In [9]:
chat(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out where to travel in one short sentence"),
        HumanMessage(content="I like the beaches where should I go?"),
        AIMessage(content="You should go to Nice, France"),
        HumanMessage(content="What else should I do when I'm there?")
    ]
)

AIMessage(content='You should explore the charming Old Town, visit the stunning Promenade des Anglais, and indulge in delicious Mediterranean cuisine.', additional_kwargs={}, example=False)

In [10]:
from langchain.schema import Document

Document(
    page_content="This is my document. It is full of text that I've gathered from other places",
    metadata={
        'my_document_id' : 234234,
        'my_document_source' : "The LangChain Papers",
        'my_document_create_time' : 1680013019
    }
)

The formula for Gravitational Potential Energy (GPE) is GPE = mgh, where m is the mass of the object, g is the acceleration due to gravity, and h is the height of the object. As an example, if a 5 kg object is dropped from a height of 10 meters, the GPE is equal to 5 x 9.81 x 10 = 490.5 J (Joules).


---
# Embeddings

In [5]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

text = "Hi! It's time for the beach"

text_embedding = embeddings.embed_query(text)
print (f"Your embedding is length {len(text_embedding)}")
print (f"Here's a sample: {text_embedding[:5]}...")

Your embedding is length 1536
Here's a sample: [-0.00011466221621958539, -0.0031506523955613375, -0.0007831145194359124, -0.019504327327013016, -0.015125557780265808]...


---
# Prompting

In [17]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

llm = OpenAI(temperature=.7)

# Notice "location" below, that is a placeholder for another value later
template = """
I really want to travel to {location}. What should I do there?

Respond in one short sentence
"""

prompt = PromptTemplate(
    input_variables=["location"],
    template=template,
)

final_prompt = prompt.format(location='Rome')

print (f"Final Prompt: {final_prompt}")
print ("-----------")
print (f"LLM Output: {llm(final_prompt)}")

Final Prompt: 
I really want to travel to Rome. What should I do there?

Respond in one short sentence

-----------
LLM Output: Visit the Colosseum, the Vatican, and the Trevi Fountain.


---
# Semantic similarity

In [2]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.7)

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input: {input}\nExample Output: {output}",
)

examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "tree", "output": "ground"},
    {"input": "bird", "output": "nest"},
]

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples, 
    
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(chunk_size=1), 
    
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS, 
    
    # This is the number of examples to produce.
    k=2
)

similar_prompt = FewShotPromptTemplate(
    # The object that will help select examples
    example_selector=example_selector,
    
    # Your prompt
    example_prompt=example_prompt,
    
    # Customizations that will be added to the top and bottom of your prompt
    prefix="Give the location an item is usually found in",
    suffix="Input: {noun}\nOutput:",
    
    # What inputs your prompt will receive
    input_variables=["noun"],
)

# Select a noun
my_noun = "student"

print(similar_prompt.format(noun=my_noun))

llm(similar_prompt.format(noun=my_noun))

Give the location an item is usually found in

Example Input: driver
Example Output: car

Example Input: pilot
Example Output: plane

Input: student
Output:


' classroom'

---
# Output formatting

In [13]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.7)

# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response"),
    ResponseSchema(name="reasoning", description="This is your response, what and why it was wrong in 1-2 sentences")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [16]:

# See the prompt template you created for formatting (default for the structured output parser, except for the passed ResponseSchema)
format_instructions = output_parser.get_format_instructions()
print (f"{format_instructions=}")

template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input="welcom to califonya!")

print(f"{promptValue=}")

llm_output = llm(promptValue)
llm_output

output_parser.parse(llm_output)

format_instructions='The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"bad_string": string  // This a poorly formatted user input string\n\t"good_string": string  // This is your response, a reformatted response\n\t"reasoning": string  // This is your response, what and why it was wrong in 1-2 sentences\n}\n```'
promptValue='\nYou will be given a poorly formatted string from a user.\nReformat it and make sure all the words are spelled correctly\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"bad_string": string  // This a poorly formatted user input string\n\t"good_string": string  // This is your response, a reformatted response\n\t"reasoning": string  // This is your response, what and why it was wrong in 1-2 sentences\n}\n```\n\n% USER INPUT:\nwelcom to califonya!\n\nYOUR R

{'bad_string': 'welcom to califonya!',
 'good_string': 'Welcome to California!',
 'reasoning': "The user misspelled 'welcome' and 'California' and added an extra 'y' to the end."}

---
# Split Document and embedd each chunck

In [ ]:

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = TextLoader('/dbfs/tmp/mbrunzel@positivethinking.tech/langchain_tutorial_data/PaulGrahamEssays/worked.txt')
documents = loader.load()

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Get embedding engine ready
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', deployment='ptc-embedding-ada-002', chunk_size=1)

# Embedd your texts
db = FAISS.from_documents(texts, embeddings)

# Init your retriever. Asking for just 1 document back
retriever = db.as_retriever()

retriever

docs = retriever.get_relevant_documents("what types of things did the author want to build?")

print("\n\n".join([x.page_content[:200] for x in docs[:2]]))


---
# Record chat history


In [ ]:
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

# chat = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)
chat = AzureChatOpenAI(
    model_name='gpt-35-turbo',
    deployment_name='ptc-chat-gpt-35',
    temperature=0.001
)

history = ChatMessageHistory()

history.add_ai_message("hi!")

history.add_user_message("what is the capital of france?")

history.messages

ai_response = chat(history.messages)
ai_response

history.add_ai_message(ai_response.content)
history.messages

---
# create templates with input variables

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

llm = AzureOpenAI(
    model_name='text-davinci-003',
    deployment_name='ptc-davinci-003',
    temperature=1,
)

template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# Holds my 'location' chain
location_chain = LLMChain(llm=llm, prompt=prompt_template)

location_chain.prompt

location_chain.run({"user_location": "Berlin"})

template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# Holds my 'meal' chain
meal_chain = LLMChain(llm=llm, prompt=prompt_template)

overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)

# Individual chain run with output of first chain
meal_chain.run('An example of a classic Roman dish would be Spaghetti Carbonara. It is a dish made with spaghetti pasta, cured pork, eggs, black pepper, and parmesan cheese.')

overall_chain.chains[1]

review = overall_chain.run("Zurich")

review